# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np

## 1.  Optimal foraging of individual agent

In [ ]:
# Read constants from config file
config_df = pd.read_csv("config.csv")
config = dict(zip(config_df["param"], config_df["value"]))

# Parameters
F = 1       # resource energy
c_s = 160
c_a = 4

# Load agent-level data
df = pd.read_csv("./simulation_results/_agents_20250604_004745.csv", delimiter=",")
df = df.apply(pd.to_numeric, errors="coerce")

# Loop over each unique resource density r
for r_value in sorted(df["resource_cap"].unique()):
    # Filter agents from this environment
    subset = df[df["resource_cap"] == r_value]

    # Find phenotypes with the highest net energy intake ε
    max_net_energy = subset["energy"].max()
    best = subset[subset["energy"] == max_net_energy]

    # Plot KDE heatmap weighted by net energy intake
    plt.figure(figsize=(7, 6))
    sns.kdeplot(
        data=subset,
        x="speed",
        y="acuity",
        weights=subset["energy"],
        fill=True,
        cmap="viridis"
    )

    # Mark maximum ε point(s)
    plt.scatter(
        best["speed"],
        best["acuity"],
        color='red',
        s=100,
        edgecolor='black',
        label="max ε"
    )

    # Labels and formatting
    plt.title(f"Net Energy Intake ε(s,a) — Resource density r = {r_value}")
    plt.xlabel("Speed")
    plt.ylabel("Acuity")
    plt.xlim(0, 0.2)
    plt.ylim(0, 0.2)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
df["C"] = c_s * df["speed"]**2 + c_a * df["acuity"]
df["mu"] = (df["energy"] + df["C"]) / df["speed"]
# Compute mean path length between encounters
df["L"] = df["speed"] / df["mu"]

# Compute L₀ (L at max acuity) for each resource density r
L0_by_r = {}

for r_value in sorted(df[resource_cap].unique()):
    group = df[df["resource_cap"] == r_value]
    max_a = group["acuity"].max()
    L0 = group[group["acuity"] == max_a]["L"].values[0]
    L0_by_r[r_value] = L0

# Plot 𝓛(a) vs. acuity for each resource density r
plt.figure(figsize=(8, 6))
sns.lineplot(data=df, x="acuity", y="L", hue="resource_cap", palette="tab10")

# Add horizontal dashed lines at each L₀
for r_value, L0 in L0_by_r.items():
    plt.axhline(L0, linestyle="--", color="gray", linewidth=1)

# Labels and formatting
plt.xlabel("Acuity")
plt.ylabel("𝓛(a) = s / ν")
plt.xlim(0, 0.2)
plt.ylim(0, 0.1)
plt.title("Mean path length between encounters vs. Acuity (grouped by resource density r)")
plt.grid(True)
plt.legend(title="Resource density (r)")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

# Loop through each resource density r
for r_value in sorted(df["resource_cap"].unique()):
    # Filter and sort by acuity
    group = df[df["resource_cap"] == r_value].sort_values("acuity")

    a = group["acuity"].values
    L = group["L"].values
    g = 1 / L
    g2 = g**2
    dg2_da = np.gradient(g2, a)

    # Plot the curve for this r
    plt.plot(a, dg2_da, label=f"r = {r_value}")

# Add horizontal dashed line at y=0
plt.axhline(0, color='gray', linestyle='--', linewidth=1, label="y = 0")

# Labels and layout
plt.xlabel("Acuity")
plt.ylabel("d(g²)/da")
plt.xlim(0, 0.2)
plt.ylim(-1, 4e4)
plt.title("1st derivative of g² vs. Acuity (by resource density r)")
plt.grid(True)
plt.legend(title="Resource density (r)")
plt.tight_layout()
plt.show()


## 2. Steady-state of a population using single strategy